In [49]:
import pandas as pd
import numpy as np

In [50]:
data = pd.read_parquet('./data/clean/positions.parquet')
data.shape

(962964, 10)

In [51]:
df = data[['Телефон_new', 'ID_SKU', 'Дата']]
df.columns = ['user', 'item', 'timestamp']

In [52]:
df.dtypes

user                 object
item                 object
timestamp    datetime64[ns]
dtype: object

In [53]:
df.isna().sum()

user         0
item         0
timestamp    0
dtype: int64

In [54]:
df.head()

,user,item,timestamp
0,55574948-52495050484877,IDL00051334048,2017-07-01
1,55574948-52495050484877,IDL00051367351,2017-07-01
2,55574948-52495050484877,IDL00007611755,2017-07-01
3,55574948-52495050484877,IDL00014478250,2017-07-01
4,55574948-52495050484877,IDL00001209351,2017-07-01


In [55]:
df['rating'] = 1

/var/folders/j7/ljxzwfz148s0_sgrdw_3_5zh0000gq/T/ipykernel_34417/3547288963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = 1


In [106]:
df = df[df['item'].isin(df['item'].value_counts()[df['item'].value_counts() > 20].index)]
df = df[df['user'].isin(df['user'].value_counts()[df['user'].value_counts() > 20].index)]

In [107]:
df.shape

(149188, 4)

In [115]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD, NMF

from surprise.model_selection import cross_validate

In [116]:
df.shape

(149188, 4)

In [117]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 2))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)

In [122]:
# Run 5-fold cross-validation and print results.
cross_validate(SVD(), data, measures=['RMSE', 'MAE'], cv=2, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    0.0496  0.0504  0.0500  0.0004  
MAE (testset)     0.0259  0.0263  0.0261  0.0002  
Fit time          3.74    3.58    3.66    0.08    
Test time         0.40    1.63    1.02    0.62    


{'test_rmse': array([0.04964144, 0.0503632 ]),
 'test_mae': array([0.0259452 , 0.02627509]),
 'fit_time': (3.738861083984375, 3.5790390968322754),
 'test_time': (0.39625024795532227, 1.6337718963623047)}